In [1]:
import os

In [2]:
%pwd

'd:\\Machine Learning\\ML projects for Resume\\Pokemon Type Detection\\Pokemon-Type-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Machine Learning\\ML projects for Resume\\Pokemon Type Detection\\Pokemon-Type-Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [8]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size
import boto3

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        self.s3_client = boto3.client('s3')


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            bucket_name, key = self._parse_s3_url(self.config.source_URL)
            self.s3_client.download_file(
                    Bucket=bucket_name,
                    Key=key,
                    Filename=self.config.local_data_file
                )
            logger.info(f"{self.config.local_data_file} download! ")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
    def _parse_s3_url(self, url):
        """Helper function to extract bucket name and key from the S3 URL."""
        if url.startswith("s3://"):
            url = url[len("s3://"):]
        bucket_name, key = url.split('/', 1)
        return bucket_name, key

In [10]:
import os
import boto3
from botocore.exceptions import NoCredentialsError
from pathlib import Path
import logging
import boto3

In [13]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-08-13 00:19:41,302: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-13 00:19:41,306: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-13 00:19:41,311: INFO: common: created directory at: artifacts]
[2024-08-13 00:19:41,311: INFO: common: created directory at: artifacts/data_ingestion]


[2024-08-13 00:27:23,991: INFO: 2165601253: artifacts/data_ingestion/data.zip download! ]
